In [ ]:
%load_ext autoreload
%autoreload 2
from common import *

In [ ]:
RESULT_JSON = "/Users/law/repos/viper/results/kv_size/all_kv_size.json"

### Rename all PmemHybrid to A_PmemHybrid (capital A) to fix the sorting

In [ ]:
FIXTURES = [VIPER, CCEH, HYBRID_FASTER, PMEMKV]
BM_TYPES = ['insert', 'get']

BMS = get_all_runs(RESULT_JSON)
# pprint(BMS) 

KV_SIZE_RE = re.compile(r".*KeyType(\d+),ValueType(\d+).*")

def get_kv_size(bm_name):
    match = KV_SIZE_RE.match(bm_name)
    if match is None:
        raise f"unknown kv size: {bm_name}"
    return (int(match.group(1)), int(match.group(2)))

RUNS = defaultdict(list)

for bm in BMS:
    for f, _ in FIXTURES:
        if f in bm['name']:            
            bm_type = get_bm_type(bm['name'])
            kv_size = get_kv_size(bm['name'])
            RUNS[(kv_size, bm_type)].append(bm)
        
    for _, run in RUNS.items():
        run.sort(key=lambda x: x['name'], reverse=True) 
        
# pprint(RUNS)

In [ ]:
fig, (insert_ax, get_ax) = plt.subplots(1, 2, figsize=(10, 3))

def get_bm_ax(bm_type):
    if bm_type == 'insert': return insert_ax
    if bm_type == 'get': return get_ax
    raise RuntimeError(f"Unknown bm_type {bm_type}")
    
KV_SIZES = sorted({kv_size for (kv_size, _) in RUNS.keys()})
SORTED_FIXTURES = [VIPER, CCEH, HYBRID_FASTER, PMEMKV]
print(SORTED_FIXTURES)

# Set position of bar on X axis
bar_width = 0.15
bar_diff = 0.02
r1 = np.arange(len(KV_SIZES))
r2 = [x + bar_width + bar_diff for x in r1]
r3 = [x + bar_width + bar_diff for x in r2]
r4 = [x + bar_width + bar_diff for x in r3]
# r5 = [x + bar_width + bar_diff for x in r4]

HATCHES = ['//', '', '\\\\', 'x']
import matplotlib as mpl
mpl.rcParams['hatch.linewidth'] = 2

for bm_type, letter in zip(BM_TYPES, ['a', 'b']):
    ax = get_bm_ax(bm_type)
#     ax.set_xlabel("Key-Value Size\n(#key bytes, #value bytes)", fontsize=20)
    ax.set_title(f"({letter}) {bm_type.upper()}", fontsize=22)

    for i, kv_size in enumerate(KV_SIZES):
        r = [run['items_per_second'] / MILLION for run in RUNS[(kv_size, bm_type)]]
        for f_id, (val, pos) in enumerate(zip(r, [r1, r2, r3, r4])):
            fixture = SORTED_FIXTURES[f_id]
            style = STYLES[fixture[0]]
            ax.bar(pos[i], val, bar_width, bottom=0, color='white', edgecolor=style.color, lw=2, hatch=style.hatch)
#             ax.bar(pos[i], val, bar_width, bottom=0, color='white', edgecolor=style.color, linewidth=2, hatch=style.hatch)
#             ax.bar(pos[i], val, bar_width, bottom=0, color=style.color, hatch=style.hatch)
#             ax.bar(pos[i], val, bar_width, bottom=0, color=style.color)
        ax.set_xticks([r + (1.6 * bar_width) for r in range(len(KV_SIZES))])
        ax.set_xticklabels([f"{ks+ vs}" for ks, vs in KV_SIZES], fontsize=26)

    ax.set_axisbelow(True)
    ax.grid(axis='y', which='major')
#     for tick in ax.xaxis.get_major_ticks():
#         tick.label.set_fontsize(17)
    for tick in ax.yaxis.get_major_ticks():
        tick.label.set_fontsize(26)


insert_ax.set_ylabel("Throughput\n(Mops/s)", fontsize=24)
insert_ax.set_title(f"(a) PUT", fontsize=22)

fig.legend(loc='upper center', labels=[f[1] for f in SORTED_FIXTURES], bbox_to_anchor=(0.503, 1.22), 
           ncol=5, frameon=False, fontsize=20, columnspacing=0.6, handletextpad=0.3)
fig.text(0.55, -0.05, 'Key-Value Record Size in Byte', ha='center', fontsize=26)


fig.text(0.169, 0.22, "X", color=STYLES[CCEH[0]].color, fontsize=20, weight='bold', family='monospace')
fig.text(0.623, 0.22, "X", color=STYLES[CCEH[0]].color, fontsize=20, weight='bold', family='monospace')

fig.text(0.202, 0.22, "X", color=STYLES[PMEMKV[0]].color, fontsize=20, weight='bold', family='monospace')
fig.text(0.66, 0.22, "X", color=STYLES[PMEMKV[0]].color, fontsize=20, weight='bold', family='monospace')

fig.savefig('charts/kv_size.pdf', bbox_inches='tight')
fig.savefig('charts/kv_size.svg', bbox_inches='tight')